In [1]:
import pickle
noise_index = 5
with open(f'/home//StepByStep/results/reasoning/combined/{noise_index}/accuracy/Activation_example_start_5_end_16.pickle', 'rb') as f:
    input_ids = pickle.load(f)

In [42]:
example = 8
with open(f'result/{noise_index}/example_{example}.pickle', 'rb') as f:
    data = pickle.load(f)



In [43]:
input_id = input_ids['input_ids'][example - 5]

In [44]:
from transformers import LlamaForCausalLM, LlamaTokenizer
MODEL_PATH = '/home/models//Llama-2-7b-hf'

tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)

In [45]:
tokens = []
for i in range(len(input_id)):
    tokens.append(f'{tokenizer.decode(input_id[i])}_{i}')


In [46]:
# save to file txt
with open(f'result/{noise_index}/example/example_{example}_tokens.txt', 'w') as f:
    for token in tokens:
        f.write(token + '\n')

In [47]:
clean_data = {}
for d in data:
    attend = []
    for d1 in d['attending_layer_head']:
        token = tokenizer.decode(input_id[d1['pos']])
        token_passed = tokenizer.decode(d1['input_id'])
        attend.append(f'layer_{d1["layer"]}_head_{d1["head"]}_token_{token}_pos_{d1["pos"]}_tokenPassed_{token_passed}')
    token = tokenizer.decode(input_id[d['pos']])
    clean_data[f'layer_{d["layer"]}_head_{d["head"]}_token_{token}_pos_{d["pos"]}'] = attend
    

In [48]:
depth_0_entry = []
for d in data:
    if d['depth'] == 0:
        depth_0_entry.append(f'layer_{d["layer"]}_head_{d["head"]}_token_{tokenizer.decode(input_id[d["pos"]])}_pos_{d["pos"]}')

In [49]:
depth_0_entry

['layer_21_head_26_token_is_pos_-1',
 'layer_21_head_27_token_is_pos_-1',
 'layer_22_head_16_token_is_pos_-1',
 'layer_31_head_27_token_is_pos_-1']

In [50]:
nested = {}

def nested_data(data, entry):
    nested_dict = {}
    new_entry = entry.split('_tokenPassed')[0]
    # print(new_entry)
    if(new_entry not in data):
        # print("Not present")
        return None
    for d in data[new_entry]:
        nested_dict[d] = nested_data(data, d)
    return nested_dict

for entry in depth_0_entry:
    nested[entry] = nested_data(clean_data, entry)



In [51]:
import json
with open(f'result/{noise_index}/example/example_{example}_nested.json', 'w') as f:
    json.dump(nested, f, indent=4, sort_keys=True)


In [52]:
# nested_data = []
# for data in clean_data:
#     for entry in data:
        


In [14]:
nested_json = {}
def create_nested_json(data):
    temp_json = {}
    for key, value in data.items():
        nested_json[key] = []
        if len(value) == 0:
            temp_json[key] = value
        else:
            nested_json[key].append(create_nested_json(value))


In [15]:
create_nested_json(clean_data)

AttributeError: 'list' object has no attribute 'items'

In [ ]:
def create_nested_dict(data_list):
    nested_dict = {}
    for entry in data_list:
        key = f"layer_{entry['layer']}_head_{entry['head']}_pos_{entry['pos']}"
        print(entry)
        if 'attending_layer_head' in entry:
            nested_dict[key] = [create_nested_dict(entry['attending_layer_head'])]
        else:
            nested_dict[key] = []
    return nested_dict

In [ ]:
create_nested_dict(clean_data)

In [ ]:
def nested_dict(node):
    result = {}
    key = f'layer_{node["layer"]}_head_{node["head"]}_pos_{node["pos"]}'
    children = node.get('attending_layer_head', [])
    if children:
        result[key] = {nested_dict(child) for child in children}
    else:
        result[key] = {}
    return result

nested_data = [nested_dict(entry) for entry in clean_data]

In [1]:
!pip install anytree


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 263.8 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [22]:
from anytree import Node, RenderTree, AsciiStyle

def create_tree(data, parent=None):
    if not isinstance(data, dict):
        return Node(str(data), parent=parent)

    current_node = Node("root") if parent is None else None
    for key, value in data.items():
        child_node = create_tree(value, parent=current_node)
        if current_node is None:
            current_node = child_node
        else:
            current_node.children += (child_node,)

    return current_node

def visualize_tree(root):
    for pre, _, node in RenderTree(root, style=AsciiStyle()):
        print(f"{pre}{node.name}")

In [23]:
data = {
    "layer_16_head_24_token_<0x0A>": {
        "layer_5_head_22_token_W": {
            "layer_0_head_22_token_<0x0A>": {},
            "layer_3_head_22_token_<0x0A>": {
                "layer_0_head_22_token_<0x0A>": {},
                "layer_0_head_15_token_<0x0A>": {}
            }
        },
        "layer_9_head_19_token_W": {
            "layer_8_head_29_token_W": {
                "layer_7_head_16_token_:": {
                    "layer_6_head_21_token_<0x0A>": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_3_head_22_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_0_head_15_token_<0x0A>": {}
                        }
                    },
                    "layer_0_head_22_token_<0x0A>": {}
                },
                "layer_3_head_16_token_:": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            },
            "layer_5_head_28_token_W": {
                "layer_0_head_22_token_<0x0A>": {},
                "layer_3_head_22_token_<0x0A>": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            }
        }
    },
    "layer_30_head_24_token_<0x0A>": {
        "layer_16_head_24_token_<0x0A>": {
            "layer_5_head_22_token_W": {
                "layer_0_head_22_token_<0x0A>": {},
                "layer_3_head_22_token_<0x0A>": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            },
            "layer_9_head_19_token_W": {
                "layer_8_head_29_token_W": {
                    "layer_7_head_16_token_:": {
                        "layer_6_head_21_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_3_head_22_token_<0x0A>": {
                                "layer_0_head_22_token_<0x0A>": {},
                                "layer_0_head_15_token_<0x0A>": {}
                            }
                        },
                        "layer_0_head_22_token_<0x0A>": {}
                    },
                    "layer_3_head_16_token_:": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_0_head_15_token_<0x0A>": {}
                    }
                },
                "layer_5_head_28_token_W": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_3_head_22_token_<0x0A>": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_0_head_15_token_<0x0A>": {}
                    }
                }
            }
        },
        "layer_18_head_30_token_<0x0A>": {
            "layer_16_head_1_token_ren": {
                "layer_12_head_17_token_is": {
                    "layer_1_head_7_token_<0x0A>": {
                        "layer_0_head_2_token_:": {},
                        "layer_0_head_31_token_:": {}
                    },
                    "layer_0_head_22_token_<0x0A>": {}
                },
                "layer_13_head_29_token_is": {
                    "layer_9_head_26_token_ren": {
                        "layer_6_head_21_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_3_head_22_token_<0x0A>": {
                                "layer_0_head_22_token_<0x0A>": {},
                                "layer_0_head_15_token_<0x0A>": {}
                            }
                        },
                        "layer_0_head_22_token_<0x0A>": {}
                    },
                    "layer_3_head_2_token_ren": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_0_head_15_token_<0x0A>": {}
                    }
                }
            },
            "layer_12_head_2_token_ren": {
                "layer_10_head_9_token_is": {
                    "layer_6_head_21_token_<0x0A>": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_3_head_22_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_0_head_15_token_<0x0A>": {}
                        }
                    },
                    "layer_0_head_22_token_<0x0A>": {}
                },
                "layer_2_head_12_token_is": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            }
        }
    }
}


In [24]:
root = create_tree(data)
visualize_tree(root)

TreeError: Cannot add non-node object None. It is not a subclass of 'NodeMixin'.

In [26]:
from anytree import Node, RenderTree, AsciiStyle

def create_tree(data, parent=None):
    if not isinstance(data, dict):
        return Node(str(data), parent=parent)

    current_node = Node("root") if parent is None else None
    for key, value in data.items():
        child_node = create_tree(value, parent=current_node)
        if current_node is None:
            current_node = child_node
        else:
            current_node.children += (child_node,)

    return current_node

def visualize_tree(root):
    for pre, _, node in RenderTree(root, style=AsciiStyle()):
        print(f"{pre}{node.name}")

data = {
    "layer_16_head_24_token_<0x0A>": {
        "layer_5_head_22_token_W": {
            "layer_0_head_22_token_<0x0A>": {},
            "layer_3_head_22_token_<0x0A>": {
                "layer_0_head_22_token_<0x0A>": {},
                "layer_0_head_15_token_<0x0A>": {}
            }
        },
        "layer_9_head_19_token_W": {
            "layer_8_head_29_token_W": {
                "layer_7_head_16_token_:": {
                    "layer_6_head_21_token_<0x0A>": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_3_head_22_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_0_head_15_token_<0x0A>": {}
                        }
                    },
                    "layer_0_head_22_token_<0x0A>": {}
                },
                "layer_3_head_16_token_:": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            },
            "layer_5_head_28_token_W": {
                "layer_0_head_22_token_<0x0A>": {},
                "layer_3_head_22_token_<0x0A>": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            }
        }
    },
    "layer_30_head_24_token_<0x0A>": {
        "layer_16_head_24_token_<0x0A>": {
            "layer_5_head_22_token_W": {
                "layer_0_head_22_token_<0x0A>": {},
                "layer_3_head_22_token_<0x0A>": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            },
            "layer_9_head_19_token_W": {
                "layer_8_head_29_token_W": {
                    "layer_7_head_16_token_:": {
                        "layer_6_head_21_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_3_head_22_token_<0x0A>": {
                                "layer_0_head_22_token_<0x0A>": {},
                                "layer_0_head_15_token_<0x0A>": {}
                            }
                        },
                        "layer_0_head_22_token_<0x0A>": {}
                    },
                    "layer_3_head_16_token_:": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_0_head_15_token_<0x0A>": {}
                    }
                },
                "layer_5_head_28_token_W": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_3_head_22_token_<0x0A>": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_0_head_15_token_<0x0A>": {}
                    }
                }
            }
        },
        "layer_18_head_30_token_<0x0A>": {
            "layer_16_head_1_token_ren": {
                "layer_12_head_17_token_is": {
                    "layer_1_head_7_token_<0x0A>": {
                        "layer_0_head_2_token_:": {},
                        "layer_0_head_31_token_:": {}
                    },
                    "layer_0_head_22_token_<0x0A>": {}
                },
                "layer_13_head_29_token_is": {
                    "layer_9_head_26_token_ren": {
                        "layer_6_head_21_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_3_head_22_token_<0x0A>": {
                                "layer_0_head_22_token_<0x0A>": {},
                                "layer_0_head_15_token_<0x0A>": {}
                            }
                        },
                        "layer_0_head_22_token_<0x0A>": {}
                    },
                    "layer_3_head_2_token_ren": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_0_head_15_token_<0x0A>": {}
                    }
                }
            },
            "layer_12_head_2_token_ren": {
                "layer_10_head_9_token_is": {
                    "layer_6_head_21_token_<0x0A>": {
                        "layer_0_head_22_token_<0x0A>": {},
                        "layer_3_head_22_token_<0x0A>": {
                            "layer_0_head_22_token_<0x0A>": {},
                            "layer_0_head_15_token_<0x0A>": {}
                        }
                    },
                    "layer_0_head_22_token_<0x0A>": {}
                },
                "layer_2_head_12_token_is": {
                    "layer_0_head_22_token_<0x0A>": {},
                    "layer_0_head_15_token_<0x0A>": {}
                }
            }
        }
    }
}

root = create_tree(data)
visualize_tree(root)


AttributeError: 'NoneType' object has no attribute 'name'